# 🚀 NeuroTrader PPO Training Farm (Dynamic)

**Purpose:** Train PPO models with **randomized hyperparameters** for Hybrid AI

**Key Feature:** Each run generates unique configs automatically!

---

## ⚠️ Before Running:
1. Go to **Runtime → Change runtime type → GPU (T4)**
2. Upload your data file when prompted

## 1. Setup

In [ ]:
!nvidia-smi
!pip install stable-baselines3[extra] gymnasium pandas pyarrow -q

import warnings
warnings.filterwarnings('ignore', message='.*Gym has been unmaintained.*')

print("\n✅ Setup complete!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
OUTPUT_DIR = '/content/drive/MyDrive/NeuroTrader_Models'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Output: {OUTPUT_DIR}")

## 2. Upload Data

In [ ]:
from google.colab import files
import shutil

print("📤 Upload XAUUSDm_M5_raw.parquet:")
uploaded = files.upload()

os.makedirs('data/raw', exist_ok=True)
for f in uploaded.keys():
    shutil.move(f, f'data/raw/{f}')
    print(f"✅ Saved: data/raw/{f}")

## 3. Dynamic Config Generator 🎲

In [ ]:
import numpy as np
import json
import time
from datetime import datetime

def generate_random_config():
    """
    Generate randomized hyperparameters for diverse model training.
    Each call produces unique values!
    """
    # Random seed based on current time (ensures uniqueness)
    seed = int(time.time() * 1000) % 1000000
    np.random.seed(seed)
    
    config = {
        'seed': seed,
        'learning_rate': np.random.choice([1e-4, 3e-4, 5e-4, 7e-4, 1e-3]),
        'n_steps': np.random.choice([512, 1024, 2048, 4096]),
        'batch_size': np.random.choice([32, 64, 128, 256]),
        'ent_coef': np.random.uniform(0.005, 0.1),
        'gamma': np.random.choice([0.95, 0.97, 0.99, 0.995]),
        'gae_lambda': np.random.uniform(0.9, 0.99),
        'clip_range': np.random.choice([0.1, 0.2, 0.3]),
        'n_epochs': np.random.choice([5, 10, 15, 20]),
        'max_grad_norm': np.random.choice([0.3, 0.5, 1.0]),
    }
    return config

# Generate config for THIS run
CONFIG = generate_random_config()
RUN_ID = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

print(f"🎲 Generated Config for {RUN_ID}:")
for k, v in CONFIG.items():
    print(f"   {k}: {v}")

## 4. Trading Environment

In [ ]:
import gymnasium as gym
import pandas as pd
from gymnasium import spaces

class SimpleTradingEnv(gym.Env):
    def __init__(self, df, lookback=20):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.lookback = lookback
        self.current_step = lookback
        self.action_space = spaces.Discrete(3)  # HOLD, BUY, SELL
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(22,), dtype=np.float32)
        self.position = 0
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.lookback
        self.position = 0
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        return self._get_obs(), {}
    
    def _get_obs(self):
        window = self.df.iloc[self.current_step-self.lookback:self.current_step]
        close = window['close'].values
        returns = np.diff(close) / (close[:-1] + 1e-8)
        obs = np.concatenate([
            returns[-10:],
            [close[-1] / close[0] - 1],
            [(close[-1] - close.min()) / (close.max() - close.min() + 1e-8)],
            [np.std(returns)],
            [self.position],
            [self.equity / 10000 - 1],
            np.zeros(7)
        ])
        return obs.astype(np.float32)
    
    def step(self, action):
        price = self.df.iloc[self.current_step]['close']
        reward = 0
        
        if action == 1 and self.position == 0:
            self.position = 1
            self.entry_price = price
            reward = 0.01
        elif action == 2 and self.position == 1:
            pnl = (price - self.entry_price) / self.entry_price
            self.balance *= (1 + pnl)
            reward = pnl * 100
            self.position = 0
        
        if self.position == 1:
            self.equity = self.balance * (1 + (price - self.entry_price) / self.entry_price)
        else:
            self.equity = self.balance
        
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        truncated = self.equity < 5000
        return self._get_obs(), reward, done, truncated, {'equity': self.equity}

print("✅ Environment ready!")

## 5. Train with Dynamic Config 🚀

In [ ]:
from stable_baselines3 import PPO
import torch

# Load data
df = pd.read_parquet('data/raw/XAUUSDm_M5_raw.parquet')
if 'tick_volume' in df.columns:
    df.rename(columns={'tick_volume': 'volume'}, inplace=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"📊 Data: {len(df)} rows | Device: {device.upper()}")

# Create environment and model with DYNAMIC config
env = SimpleTradingEnv(df)

model = PPO(
    "MlpPolicy",
    env,
    device=device,
    verbose=1,
    seed=CONFIG['seed'],
    learning_rate=CONFIG['learning_rate'],
    n_steps=int(CONFIG['n_steps']),
    batch_size=int(CONFIG['batch_size']),
    ent_coef=CONFIG['ent_coef'],
    gamma=CONFIG['gamma'],
    gae_lambda=CONFIG['gae_lambda'],
    clip_range=CONFIG['clip_range'],
    n_epochs=int(CONFIG['n_epochs']),
    max_grad_norm=CONFIG['max_grad_norm']
)

print(f"\n🚀 Training {RUN_ID}...")
start = datetime.now()
model.learn(total_timesteps=500_000, progress_bar=True)
duration = (datetime.now() - start).total_seconds()

# Save model
save_path = f"{OUTPUT_DIR}/ppo_{RUN_ID}"
model.save(save_path)
print(f"\n✅ Saved: {save_path}.zip ({duration/60:.1f} min)")

## 6. Evaluate & Log

In [ ]:
# Quick evaluation
env_test = SimpleTradingEnv(df[-5000:])
obs, _ = env_test.reset()
trades = 0

for _ in range(4000):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, done, trunc, info = env_test.step(action)
    if action in [1, 2]: trades += 1
    if done or trunc: break

final_return = (info['equity'] / 10000 - 1) * 100
print(f"📈 Result: {final_return:+.2f}% | {trades} trades")

# Save to training log
log_path = f"{OUTPUT_DIR}/training_log.json"
log_entry = {
    'run_id': RUN_ID,
    'config': {k: float(v) if isinstance(v, (np.floating, np.integer)) else v for k, v in CONFIG.items()},
    'result': {'return_pct': final_return, 'trades': trades},
    'duration_min': duration/60,
    'timestamp': datetime.now().isoformat()
}

# Append to log
try:
    with open(log_path, 'r') as f:
        logs = json.load(f)
except:
    logs = []
logs.append(log_entry)
with open(log_path, 'w') as f:
    json.dump(logs, f, indent=2)

print(f"📝 Logged to: {log_path}")
print(f"\n🔄 To train another model with NEW config: Runtime → Run all")

## 7. View Training History

In [ ]:
# Show all trained models
try:
    with open(log_path, 'r') as f:
        logs = json.load(f)
    
    print(f"📊 Training History ({len(logs)} models):")
    print("-" * 60)
    for log in logs:
        r = log['result']
        print(f"{log['run_id']}: {r['return_pct']:+.2f}% | {r['trades']} trades | LR={log['config']['learning_rate']:.0e}")
except:
    print("No training history yet.")